In [1]:
# set SLM

from meadowlark import Meadowlark
import numpy as np

slm = Meadowlark(
    verbose=True,
    sdk_path="C:\\Program Files\\Meadowlark Optics\\Blink OverDrive Plus",
    # lut_path="C:\\Program Files\\Meadowlark Optics\\SDK\\slm5691_at635.LUT",
    lut_path="C:\\Program Files\\Meadowlark Optics\\SDK\\1920x1152_linearVoltage.LUT",
    wav_um=0.550,
    pitch_um=(9.2, 9.2),
)

print(slm.shape)



Validating DPI awareness...success
Loading Blink SDK libraries...success
Initializing SDK...success
Found 1 SLM controller(s)
Loading LUT file...success
(1152, 1920)


In [2]:


def generate_and_load_fresnel_lens(slm, focal_length_mm, angle_x_mrad, angle_y_mrad, 
                                   wavelength=550e-9, pixel_size=9.2e-6, two_pi_value=190):
    """
    Generate and load a Fresnel lens phase pattern with beam steering to SLM.
    
    Parameters:
    -----------
    slm : object
        The SLM object with .shape attribute and .set_phase() method
    focal_length_mm : float
        Focal length in millimeters (positive for converging, negative for diverging)
    angle_x_mrad : float
        Steering angle in x direction in milliradians
    angle_y_mrad : float
        Steering angle in y direction in milliradians
    wavelength : float
        Wavelength of light in meters (default: 550nm)
    pixel_size : float
        Size of the SLM pixel in meters (default: 9.2μm)
    two_pi_value : int
        SLM grayscale value corresponding to 2π phase shift (default: 190)
    
    Returns:
    --------
    phase_uint8 : ndarray
        The phase pattern as uint8 array that was loaded to the SLM
    """
    # Get SLM shape
    height, width = slm.shape
    
    # Convert focal length to meters
    focal_length_m = focal_length_mm * 1e-3
    
    # Create coordinate grid
    y, x = np.indices((height, width))
    
    # Calculate center of the SLM
    center_x = width / 2
    center_y = height / 2
    
    # Calculate distances from center in meters
    x_meters = (x - center_x) * pixel_size
    y_meters = (y - center_y) * pixel_size
    r_squared = x_meters**2 + y_meters**2
    
    # Calculate Fresnel lens phase
    # φ(x,y) = (2π/λ) * (f - sqrt(f² + x² + y²))
    f_squared = focal_length_m**2
    lens_phase = (2 * np.pi / wavelength) * (focal_length_m - np.sqrt(f_squared + r_squared))
    
    # Calculate linear phase for beam steering
    # Convert milliradians to radians
    angle_x_rad = -angle_x_mrad * 1e-3
    angle_y_rad = -angle_y_mrad * 1e-3
    
    # Phase gradients for steering
    phase_gradient_x = 2 * np.pi * np.sin(angle_x_rad) / wavelength
    phase_gradient_y = 2 * np.pi * np.sin(angle_y_rad) / wavelength
    
    # Linear phase for beam steering
    steering_phase = phase_gradient_x * x_meters + phase_gradient_y * y_meters
    
    # Combine lens and steering phases
    total_phase = lens_phase + steering_phase
    
    # Wrap phase to [0, 2π) range
    total_phase = total_phase % (2 * np.pi)
    
    # Convert to uint8 for SLM
    phase_uint8 = np.uint8(total_phase / (2 * np.pi) * two_pi_value)
    
    # Load to SLM
    slm.set_phase(phase_uint8,settle=True)
    
    return phase_uint8



In [ ]:
import cv2
import time
import numpy as np

# 加载预训练的人脸检测器
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 打开摄像头
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# 相机参数 (根据你的相机调整这些值)
FOV_HORIZONTAL = 4  # 水平视场角（度）
FOV_VERTICAL = 3    # 垂直视场角（度）

# 获取帧尺寸
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 计算图像中心
center_x = frame_width // 2
center_y = frame_height // 2

# FPS计算
fps_time = 0
fps_counter = 0
fps_display = 0

def calculate_angular_position(face_center_x, face_center_y, img_width, img_height, fov_h, fov_v):
    """
    计算人脸中心相对于图像中心的角度位置
    
    参数:
    - face_center_x, face_center_y: 人脸中心坐标
    - img_width, img_height: 图像尺寸
    - fov_h, fov_v: 水平和垂直视场角（度）
    
    返回:
    - theta_x: 水平角度（正值表示右侧，负值表示左侧）
    - theta_y: 垂直角度（正值表示上方，负值表示下方）
    """
    # 图像中心
    img_center_x = img_width / 2
    img_center_y = img_height / 2
    
    # 计算相对位置（归一化到-1到1）
    rel_x = (face_center_x - img_center_x) / (img_width / 2)
    rel_y = -(face_center_y - img_center_y) / (img_height / 2)  # Y轴反转
    
    # 计算角度
    theta_x = rel_x * (fov_h / 2)
    theta_y = rel_y * (fov_v / 2)
    
    return theta_x, theta_y

def draw_crosshair(img, x, y, size=20, color=(0, 255, 255), thickness=2):
    """绘制十字准心"""
    cv2.line(img, (x - size, y), (x + size, y), color, thickness)
    cv2.line(img, (x, y - size), (x, y + size), color, thickness)

print("按 'q' 退出程序")
print(f"相机FOV设置: {FOV_HORIZONTAL}° (水平) x {FOV_VERTICAL}° (垂直)")
print(f"分辨率: {frame_width} x {frame_height}")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 转换为灰度图（Haar Cascade在灰度图上工作）
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 检测人脸
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=10,
        minSize=(80, 80)
    )
    
    # 绘制图像中心十字准心
    draw_crosshair(frame, center_x, center_y, size=30, color=(128, 128, 128), thickness=1)
    
    # 处理每个检测到的人脸
    for i, (x, y, w, h) in enumerate(faces):
        # 绘制人脸边界框
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # 计算人脸中心
        face_center_x = x + w // 2
        face_center_y = y + h // 2
        
        # 绘制人脸中心点
        cv2.circle(frame, (face_center_x, face_center_y), 5, (0, 0, 255), -1)
        
        # 计算角度位置
        theta_x, theta_y = calculate_angular_position(
            face_center_x, face_center_y,
            frame_width, frame_height,
            FOV_HORIZONTAL, FOV_VERTICAL
        )
        
        # 计算相对位置（百分比）
        rel_x_percent = ((face_center_x - center_x) / center_x) * 100
        rel_y_percent = -((face_center_y - center_y) / center_y) * 100  # Y轴反转
        
        # 显示人脸信息
        info_y = y - 10 if y > 100 else y + h + 20
        
        # 角度信息
        angle_text = f"Angle: ({theta_x:.1f}, {theta_y:.1f})"
        cv2.putText(frame, angle_text, (x, info_y),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        
        # 相对位置信息
        pos_text = f"Pos: ({rel_x_percent:.0f}%, {rel_y_percent:.0f}%)"
        cv2.putText(frame, pos_text, (x, info_y + 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        
        # 绘制从图像中心到人脸中心的线
        cv2.line(frame, (center_x, center_y), (face_center_x, face_center_y), 
                (0, 255, 255), 1)
    
    # 计算FPS
    fps_counter += 1
    if time.time() - fps_time > 1:
        fps_display = fps_counter
        fps_counter = 0
        fps_time = time.time()
    
    # 显示信息面板
    panel_y = 30
    cv2.putText(frame, f'FPS: {fps_display}', (10, panel_y),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
    cv2.putText(frame, f'Faces: {len(faces)}', (10, panel_y + 25),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)
    
    
    # 绘制坐标轴说明（右上角）
    legend_x = frame_width - 150
    legend_y = 30
    cv2.putText(frame, 'Center: (0, 0)', (legend_x, legend_y),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    cv2.putText(frame, '+X: Right', (legend_x, legend_y + 20),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    cv2.putText(frame, '+Y: Up', (legend_x, legend_y + 40),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    
    # 显示结果
    cv2.imshow('Face Tracking with Angular Position', frame)
    
    # 按键处理
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('h'):  # 增加水平FOV
        FOV_HORIZONTAL = min(FOV_HORIZONTAL + 5, 180)
        print(f"FOV水平: {FOV_HORIZONTAL}°")
    elif key == ord('g'):  # 减少水平FOV
        FOV_HORIZONTAL = max(FOV_HORIZONTAL - 5, 10)
        print(f"FOV水平: {FOV_HORIZONTAL}°")
    elif key == ord('v'):  # 增加垂直FOV
        FOV_VERTICAL = min(FOV_VERTICAL + 5, 180)
        print(f"FOV垂直: {FOV_VERTICAL}°")
    elif key == ord('b'):  # 减少垂直FOV
        FOV_VERTICAL = max(FOV_VERTICAL - 5, 10)
        print(f"FOV垂直: {FOV_VERTICAL}°")

print("\n程序结束")
print("按键说明:")
print("  q - 退出")
print("  h/g - 增加/减少水平FOV")
print("  v/b - 增加/减少垂直FOV")

cap.release()
cv2.destroyAllWindows()

按 'q' 退出程序
相机FOV设置: 15° (水平) x 10° (垂直)
分辨率: 1024 x 576

程序结束
按键说明:
  q - 退出
  h/g - 增加/减少水平FOV
  v/b - 增加/减少垂直FOV


In [3]:
import cv2
import time
import numpy as np
import math

# --- Your face detector as-is ---
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# --- Camera setup ---
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# --- Camera FOV (degrees); tune to your camera ---
FOV_HORIZONTAL = 4  # deg
FOV_VERTICAL   = 3  # deg

# --- Frame size and center ---
frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
center_x = frame_width // 2
center_y = frame_height // 2

# --- FPS book-keeping ---
fps_time = 0
fps_counter = 0
fps_display = 0

# ===================== SLM CONTROL PARAMS =====================
# Provide / import these in your environment:
# from your_slm_module import generate_and_load_fresnel_lens
# slm = <your SLM handle>

FOCAL_LENGTH_MM = 237  # fixed lens; change to 238 if that's your calibrated value
# Search (when no face)
SCAN_RANGE_MRAD = 18.0     # +/- range each axis for raster (mrad)
SCAN_STEP_MRAD  = 2.0      # step per raster cell (mrad)
# Tracking (when face is detected)
KP = 0.7                  # proportional gain (<1), maps camera angle (deg) -> SLM step (mrad)
MAX_STEP_PER_FRAME_MRAD = 12.0  # clamp how much we move per frame
CENTER_DEADBAND_DEG = 0.01     # if |error| < deadband, treat as centered
# Safety clamp on absolute steering (keep within scan range unless you know SLM limits)
MAX_ABS_MRAD = SCAN_RANGE_MRAD

# If your SLM axis directions are reversed relative to camera, flip here:
ANGLE_SIGN_X = +1.0  # set to -1.0 if steering goes the wrong way horizontally
ANGLE_SIGN_Y = +1.0  # set to -1.0 if steering goes the wrong way vertically

# State: commanded angles currently loaded on SLM (mrad)
slm_angle_x = 0.0
slm_angle_y = 0.0

# Build a raster scan path (snake pattern), centered at (0,0)
def build_raster_positions(max_mrad=10.0, step=2.0):
    n = int(round(2*max_mrad/step)) + 1
    ys = np.linspace(-max_mrad, max_mrad, n)
    xs = np.linspace(-max_mrad, max_mrad, n)
    path = []
    for r, y in enumerate(ys):
        row_xs = xs if r % 2 == 0 else xs[::-1]
        for x in row_xs:
            path.append((float(x), float(y)))
    return path

raster_path = build_raster_positions(SCAN_RANGE_MRAD, SCAN_STEP_MRAD)
raster_idx = 0
search_mode = True  # start in search until we see a face

def deg_to_mrad(d):
    return d * (math.pi/180.0) * 1000.0

def calculate_angular_position(face_center_x, face_center_y, img_width, img_height, fov_h, fov_v):
    img_center_x = img_width / 2
    img_center_y = img_height / 2
    rel_x = (face_center_x - img_center_x) / (img_width / 2)
    rel_y = -(face_center_y - img_center_y) / (img_height / 2)  # invert Y axis
    theta_x = rel_x * (fov_h / 2)
    theta_y = rel_y * (fov_v / 2)
    return theta_x, theta_y

def draw_crosshair(img, x, y, size=20, color=(0, 255, 255), thickness=2):
    cv2.line(img, (x - size, y), (x + size, y), color, thickness)
    cv2.line(img, (x, y - size), (x, y + size), color, thickness)

print("按 'q' 退出程序")
print(f"相机FOV设置: {FOV_HORIZONTAL}° (水平) x {FOV_VERTICAL}° (垂直)")
print(f"分辨率: {frame_width} x {frame_height}")
print("键盘: [ / ] 调整Kp,  ,/. 调整MAX_STEP, r复位SLM, x/y翻转轴符号, s切换搜索, p暂停SLM更新")

slm_updates_enabled = True  # 'p' to toggle
update_flag = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=10,
        minSize=(80, 80)
    )

    draw_crosshair(frame, center_x, center_y, size=30, color=(128, 128, 128), thickness=1)

    # Draw all faces; choose primary (largest) for control
    primary_face = None
    if len(faces) > 0:
        primary_face = max(faces, key=lambda r: r[2]*r[3])
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cx, cy = x + w//2, y + h//2
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

    # Decide ONE SLM update per loop
    want_update = False
    target_x_mrad = slm_angle_x
    target_y_mrad = slm_angle_y
    mode_text = "SEARCH"

    if primary_face is None:
        # SEARCH: one raster step per frame
        search_mode = True
        x_mrad, y_mrad = raster_path[raster_idx]
        raster_idx = (raster_idx + 1) % len(raster_path)
        target_x_mrad = x_mrad
        target_y_mrad = y_mrad
        want_update = True
    else:
        # TRACK: proportional correction towards center
        search_mode = False
        (x, y, w, h) = primary_face
        face_cx = x + w//2
        face_cy = y + h//2

        theta_x_deg, theta_y_deg = calculate_angular_position(
            face_cx, face_cy,
            frame_width, frame_height,
            FOV_HORIZONTAL, FOV_VERTICAL
        )

        # Overlay for the primary face
        info_y = (y - 10) if y > 100 else (y + h + 20)
        angle_text = f"Angle: ({theta_x_deg:.2f} deg, {theta_y_deg:.2f} deg)"
        cv2.putText(frame, angle_text, (x, info_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        pos_text = f"Pos: ({int((face_cx-center_x)/center_x*100)}%, {int(-(face_cy-center_y)/center_y*100)}%)"
        cv2.putText(frame, pos_text, (x, info_y + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

        # Deadband to avoid chattering
        if (abs(theta_x_deg) < CENTER_DEADBAND_DEG) and (abs(theta_y_deg) < CENTER_DEADBAND_DEG):
            mode_text = "CENTERED"
        else:
            mode_text = "TRACK"
            # Convert camera error (deg) -> desired SLM step (mrad)
            step_x_mrad = ANGLE_SIGN_X * KP * deg_to_mrad(theta_x_deg)
            step_y_mrad = ANGLE_SIGN_Y * KP * deg_to_mrad(theta_y_deg)
            # Clamp per-frame step
            step_mag = max(abs(step_x_mrad), abs(step_y_mrad))
            if step_mag > MAX_STEP_PER_FRAME_MRAD:
                scale = MAX_STEP_PER_FRAME_MRAD / (step_mag + 1e-9)
                step_x_mrad *= scale
                step_y_mrad *= scale
            # Update targets
            target_x_mrad = np.clip(slm_angle_x + step_x_mrad, -MAX_ABS_MRAD, +MAX_ABS_MRAD)
            target_y_mrad = np.clip(slm_angle_y + step_y_mrad, -MAX_ABS_MRAD, +MAX_ABS_MRAD)
            want_update = True

        # Draw line from image center to chosen face center
        cv2.line(frame, (center_x, center_y), (face_cx, face_cy), (0, 255, 255), 1)

    # ONE SLM CALL PER LOOP (if needed + enabled)
    if want_update and slm_updates_enabled and update_flag==0:
        try:
            # IMPORTANT: single call per iteration
            _ = generate_and_load_fresnel_lens(
                slm,
                focal_length_mm=FOCAL_LENGTH_MM,
                angle_x_mrad=float(target_x_mrad),
                angle_y_mrad=float(target_y_mrad)
            )
            slm_angle_x = target_x_mrad
            slm_angle_y = target_y_mrad
        except Exception as e:
            # Keep UI running even if SLM call fails
            cv2.putText(frame, f"SLM ERR: {str(e)[:40]}", (10, frame_height - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        # update every 2 frames
        update_flag = update_flag + 1
    else:
        update_flag = 0
    if update_flag>=2:
        update_flag = 0
    
    # FPS update
    fps_counter += 1
    if time.time() - fps_time > 1:
        fps_display = fps_counter
        fps_counter = 0
        fps_time = time.time()

    # HUD / legend
    panel_y = 30
    cv2.putText(frame, f'FPS: {fps_display}', (10, panel_y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    cv2.putText(frame, f'Faces: {len(faces)}', (10, panel_y + 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)

    legend_x = frame_width - 220
    legend_y = 30
    cv2.putText(frame, 'Center: (0, 0)', (legend_x, legend_y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    cv2.putText(frame, '+X: Right, +Y: Up', (legend_x, legend_y + 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)

    # SLM status
    mode_color = (0,255,0) if mode_text=="CENTERED" else ((0,165,255) if mode_text=="TRACK" else (255,255,0))
    cv2.putText(frame, f'Mode: {mode_text}', (10, panel_y + 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, mode_color, 2)
    cv2.putText(frame, f'SLM (mrad): x={slm_angle_x:.2f}, y={slm_angle_y:.2f}', (10, panel_y + 75),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)
    cv2.putText(frame, f'Kp={KP:.2f}, MaxStep={MAX_STEP_PER_FRAME_MRAD:.2f} mrad', (10, panel_y + 95),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)
    cv2.putText(frame, f'SLM {"ON" if slm_updates_enabled else "PAUSED"}  FL={FOCAL_LENGTH_MM} mm', (10, panel_y + 115),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 255, 200), 1)

    cv2.imshow('Face Tracking + SLM Control', frame)

    # Keys
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('h'):
        FOV_HORIZONTAL = min(FOV_HORIZONTAL + 5, 180); print(f"FOV水平: {FOV_HORIZONTAL}°")
    elif key == ord('g'):
        FOV_HORIZONTAL = max(FOV_HORIZONTAL - 5, 1);   print(f"FOV水平: {FOV_HORIZONTAL}°")
    elif key == ord('v'):
        FOV_VERTICAL   = min(FOV_VERTICAL + 5, 180);   print(f"FOV垂直: {FOV_VERTICAL}°")
    elif key == ord('b'):
        FOV_VERTICAL   = max(FOV_VERTICAL - 5, 1);     print(f"FOV垂直: {FOV_VERTICAL}°")
    elif key == ord('['):  # decrease Kp
        KP = max(0.05, KP - 0.05); print(f"Kp -> {KP:.2f}")
    elif key == ord(']'):  # increase Kp
        KP = min(0.95, KP + 0.05); print(f"Kp -> {KP:.2f}")
    elif key == ord(','):  # decrease max step
        MAX_STEP_PER_FRAME_MRAD = max(0.1, round(MAX_STEP_PER_FRAME_MRAD - 0.1, 2)); print(f"MaxStep -> {MAX_STEP_PER_FRAME_MRAD:.2f} mrad")
    elif key == ord('.'):  # increase max step
        MAX_STEP_PER_FRAME_MRAD = min(5.0, round(MAX_STEP_PER_FRAME_MRAD + 0.1, 2)); print(f"MaxStep -> {MAX_STEP_PER_FRAME_MRAD:.2f} mrad")
    elif key == ord('r'):  # reset SLM to center
        slm_angle_x = 0.0; slm_angle_y = 0.0
        try:
            _ = generate_and_load_fresnel_lens(slm, focal_length_mm=FOCAL_LENGTH_MM,
                                               angle_x_mrad=0.0, angle_y_mrad=0.0)
        except Exception as e:
            print("Reset SLM error:", e)
    elif key == ord('x'):  # flip X sign
        ANGLE_SIGN_X *= -1.0; print(f"ANGLE_SIGN_X -> {ANGLE_SIGN_X:+.0f}")
    elif key == ord('y'):  # flip Y sign
        ANGLE_SIGN_Y *= -1.0; print(f"ANGLE_SIGN_Y -> {ANGLE_SIGN_Y:+.0f}")
    elif key == ord('s'):  # toggle search mode (forces next-frame raster step if no face)
        search_mode = True; print("Force SEARCH mode when no face")
    elif key == ord('p'):  # pause/resume SLM updates (for testing)
        slm_updates_enabled = not slm_updates_enabled
        print("SLM updates:", "ENABLED" if slm_updates_enabled else "PAUSED")

print("\n程序结束")
cap.release()
cv2.destroyAllWindows()


按 'q' 退出程序
相机FOV设置: 4° (水平) x 3° (垂直)
分辨率: 1024 x 576
键盘: [ / ] 调整Kp,  ,/. 调整MAX_STEP, r复位SLM, x/y翻转轴符号, s切换搜索, p暂停SLM更新

程序结束


In [ ]:
import cv2
import time
import numpy as np
import math
from collections import deque

# --- Your face detector as-is ---
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# --- Camera setup ---
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# --- Camera FOV (degrees); tune to your camera ---
FOV_HORIZONTAL = 4  # deg
FOV_VERTICAL   = 3  # deg

# --- Frame size and center ---
frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
center_x = frame_width // 2
center_y = frame_height // 2

# --- FPS book-keeping ---
fps_time = 0
fps_counter = 0
fps_display = 0

# ===================== SLM CONTROL PARAMS =====================
# Provide / import these in your environment:
# from your_slm_module import generate_and_load_fresnel_lens
# slm = <your SLM handle>

FOCAL_LENGTH_MM = 237  # fixed lens; change to 238 if that's your calibrated value
# Search (when no face)
SCAN_RANGE_MRAD = 20.0     # +/- range each axis for raster (mrad)
SCAN_STEP_MRAD  = 2.0      # step per raster cell (mrad)

# ===================== PID CONTROL PARAMETERS =====================
# PID gains
KP = 0.4                   # proportional gain
KI = 0.1                  # integral gain
KD = 0.01                  # derivative gain

# PID tuning helpers
MAX_STEP_PER_FRAME_MRAD = 10.0  # clamp how much we move per frame
CENTER_DEADBAND_DEG = 0.005       # if |error| < deadband, treat as centered
INTEGRAL_WINDUP_LIMIT = 10.0     # limit integral accumulation (mrad)
DERIVATIVE_FILTER_ALPHA = 0.3    # low-pass filter for derivative (0-1, lower = more filtering)

# Safety clamp on absolute steering (keep within scan range unless you know SLM limits)
MAX_ABS_MRAD = SCAN_RANGE_MRAD

# If your SLM axis directions are reversed relative to camera, flip here:
ANGLE_SIGN_X = +1.0  # set to -1.0 if steering goes the wrong way horizontally
ANGLE_SIGN_Y = +1.0  # set to -1.0 if steering goes the wrong way vertically

# ===================== PID STATE VARIABLES =====================
# State: commanded angles currently loaded on SLM (mrad)
slm_angle_x = 0.0
slm_angle_y = 0.0

# PID error tracking
integral_x = 0.0
integral_y = 0.0
prev_error_x = 0.0
prev_error_y = 0.0
filtered_deriv_x = 0.0
filtered_deriv_y = 0.0
last_update_time = time.time()

# Error history for visualization (optional)
error_history_x = deque(maxlen=100)
error_history_y = deque(maxlen=100)

# Build a raster scan path (snake pattern), centered at (0,0)
def build_raster_positions(max_mrad=10.0, step=2.0):
    n = int(round(2*max_mrad/step)) + 1
    ys = np.linspace(-max_mrad, max_mrad, n)
    xs = np.linspace(-max_mrad, max_mrad, n)
    path = []
    for r, y in enumerate(ys):
        row_xs = xs if r % 2 == 0 else xs[::-1]
        for x in row_xs:
            path.append((float(x), float(y)))
    return path

raster_path = build_raster_positions(SCAN_RANGE_MRAD, SCAN_STEP_MRAD)
raster_idx = 0
search_mode = True  # start in search until we see a face

def deg_to_mrad(d):
    return d * (math.pi/180.0) * 1000.0

def calculate_angular_position(face_center_x, face_center_y, img_width, img_height, fov_h, fov_v):
    img_center_x = img_width / 2
    img_center_y = img_height / 2
    rel_x = (face_center_x - img_center_x) / (img_width / 2)
    rel_y = -(face_center_y - img_center_y) / (img_height / 2)  # invert Y axis
    theta_x = rel_x * (fov_h / 2)
    theta_y = rel_y * (fov_v / 2)
    return theta_x, theta_y

def draw_crosshair(img, x, y, size=20, color=(0, 255, 255), thickness=2):
    cv2.line(img, (x - size, y), (x + size, y), color, thickness)
    cv2.line(img, (x, y - size), (x, y + size), color, thickness)

def reset_pid_state():
    """Reset PID controller state"""
    global integral_x, integral_y, prev_error_x, prev_error_y, filtered_deriv_x, filtered_deriv_y
    integral_x = 0.0
    integral_y = 0.0
    prev_error_x = 0.0
    prev_error_y = 0.0
    filtered_deriv_x = 0.0
    filtered_deriv_y = 0.0
    error_history_x.clear()
    error_history_y.clear()

def compute_pid_control(error_deg, prev_error, integral, filtered_deriv, dt, axis='x'):
    """Compute PID control output for one axis"""
    global KP, KI, KD, INTEGRAL_WINDUP_LIMIT, DERIVATIVE_FILTER_ALPHA
    
    # Convert error from degrees to mrad
    error_mrad = deg_to_mrad(error_deg)
    prev_error_mrad = deg_to_mrad(prev_error)
    
    # Proportional term
    p_term = KP * error_mrad
    
    # Integral term with anti-windup
    integral += error_mrad * dt
    integral = np.clip(integral, -INTEGRAL_WINDUP_LIMIT, INTEGRAL_WINDUP_LIMIT)
    i_term = KI * integral
    
    # Derivative term with filtering
    if dt > 0:
        raw_deriv = (error_mrad - prev_error_mrad) / dt
        filtered_deriv = DERIVATIVE_FILTER_ALPHA * raw_deriv + (1 - DERIVATIVE_FILTER_ALPHA) * filtered_deriv
    else:
        filtered_deriv = 0.0
    d_term = KD * filtered_deriv
    
    # Combined PID output
    pid_output = p_term + i_term + d_term
    
    # Apply axis sign
    if axis == 'x':
        pid_output *= ANGLE_SIGN_X
    else:
        pid_output *= ANGLE_SIGN_Y
    
    return pid_output, integral, filtered_deriv

print("按 'q' 退出程序")
print(f"相机FOV设置: {FOV_HORIZONTAL}° (水平) x {FOV_VERTICAL}° (垂直)")
print(f"分辨率: {frame_width} x {frame_height}")
print("\n=== PID控制键盘快捷键 ===")
print("P增益: [ / ] 调整Kp")
print("I增益: i / o 调整Ki")
print("D增益: k / l 调整Kd")
print("其他: ,/. 调整MAX_STEP, r复位SLM+PID, x/y翻转轴符号")
print("     s切换搜索, p暂停SLM更新, c清除积分项")
print(f"\n初始PID参数: Kp={KP:.2f}, Ki={KI:.2f}, Kd={KD:.2f}")

slm_updates_enabled = True  # 'p' to toggle
update_flag = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    current_time = time.time()
    dt = current_time - last_update_time
    last_update_time = current_time

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=10,
        minSize=(80, 80)
    )

    draw_crosshair(frame, center_x, center_y, size=30, color=(128, 128, 128), thickness=1)

    # Draw all faces; choose primary (largest) for control
    primary_face = None
    if len(faces) > 0:
        primary_face = max(faces, key=lambda r: r[2]*r[3])
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cx, cy = x + w//2, y + h//2
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

    # Decide ONE SLM update per loop
    want_update = False
    target_x_mrad = slm_angle_x
    target_y_mrad = slm_angle_y
    mode_text = "SEARCH"

    if primary_face is None:
        # SEARCH: one raster step per frame
        search_mode = True
        # Reset PID state when losing track
        if prev_error_x != 0 or prev_error_y != 0:
            reset_pid_state()
        
        x_mrad, y_mrad = raster_path[raster_idx]
        raster_idx = (raster_idx + 1) % len(raster_path)
        target_x_mrad = x_mrad
        target_y_mrad = y_mrad
        want_update = True
    else:
        # TRACK: PID control towards center
        search_mode = False
        (x, y, w, h) = primary_face
        face_cx = x + w//2
        face_cy = y + h//2

        theta_x_deg, theta_y_deg = calculate_angular_position(
            face_cx, face_cy,
            frame_width, frame_height,
            FOV_HORIZONTAL, FOV_VERTICAL
        )
        
        # Store error history for visualization
        error_history_x.append(theta_x_deg)
        error_history_y.append(theta_y_deg)

        # Overlay for the primary face
        info_y = (y - 10) if y > 100 else (y + h + 20)
        angle_text = f"Error: ({theta_x_deg:.2f}deg, {theta_y_deg:.2f}deg)"
        cv2.putText(frame, angle_text, (x, info_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        pos_text = f"Pos: ({int((face_cx-center_x)/center_x*100)}%, {int(-(face_cy-center_y)/center_y*100)}%)"
        cv2.putText(frame, pos_text, (x, info_y + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

        # Deadband to avoid chattering
        if (abs(theta_x_deg) < CENTER_DEADBAND_DEG) and (abs(theta_y_deg) < CENTER_DEADBAND_DEG):
            mode_text = "CENTERED"
            # Optionally reduce integral when centered to prevent overshoot
            integral_x *= 0.95
            integral_y *= 0.95
        else:
            mode_text = "TRACK"
            
            # PID control calculation
            step_x_mrad, integral_x, filtered_deriv_x = compute_pid_control(
                theta_x_deg, prev_error_x, integral_x, filtered_deriv_x, dt, 'x'
            )
            step_y_mrad, integral_y, filtered_deriv_y = compute_pid_control(
                theta_y_deg, prev_error_y, integral_y, filtered_deriv_y, dt, 'y'
            )
            
            # Clamp per-frame step
            step_mag = max(abs(step_x_mrad), abs(step_y_mrad))
            if step_mag > MAX_STEP_PER_FRAME_MRAD:
                scale = MAX_STEP_PER_FRAME_MRAD / (step_mag + 1e-9)
                step_x_mrad *= scale
                step_y_mrad *= scale
            
            # Update targets
            target_x_mrad = np.clip(slm_angle_x + step_x_mrad, -MAX_ABS_MRAD, +MAX_ABS_MRAD)
            target_y_mrad = np.clip(slm_angle_y + step_y_mrad, -MAX_ABS_MRAD, +MAX_ABS_MRAD)
            want_update = True
            
            # Update previous errors
            prev_error_x = theta_x_deg
            prev_error_y = theta_y_deg

        # Draw line from image center to chosen face center
        cv2.line(frame, (center_x, center_y), (face_cx, face_cy), (0, 255, 255), 1)

    # ONE SLM CALL PER LOOP (if needed + enabled)
    if want_update and slm_updates_enabled and update_flag==0:
        try:
            # IMPORTANT: single call per iteration
            _ = generate_and_load_fresnel_lens(
                slm,
                focal_length_mm=FOCAL_LENGTH_MM,
                angle_x_mrad=float(target_x_mrad),
                angle_y_mrad=float(target_y_mrad)
            )
            slm_angle_x = target_x_mrad
            slm_angle_y = target_y_mrad
        except Exception as e:
            # Keep UI running even if SLM call fails
            cv2.putText(frame, f"SLM ERR: {str(e)[:40]}", (10, frame_height - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        # update every 2 frames
        update_flag = update_flag + 1
    else:
        update_flag = 0
    if update_flag>=2:
        update_flag = 0
    
    # FPS update
    fps_counter += 1
    if time.time() - fps_time > 1:
        fps_display = fps_counter
        fps_counter = 0
        fps_time = time.time()

    # HUD / legend
    panel_y = 30
    cv2.putText(frame, f'FPS: {fps_display}', (10, panel_y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    cv2.putText(frame, f'Faces: {len(faces)}', (10, panel_y + 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)

    legend_x = frame_width - 220
    legend_y = 30
    cv2.putText(frame, 'Center: (0, 0)', (legend_x, legend_y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    cv2.putText(frame, '+X: Right, +Y: Up', (legend_x, legend_y + 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)

    # SLM status
    mode_color = (0,255,0) if mode_text=="CENTERED" else ((0,165,255) if mode_text=="TRACK" else (255,255,0))
    cv2.putText(frame, f'Mode: {mode_text}', (10, panel_y + 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, mode_color, 2)
    cv2.putText(frame, f'SLM (mrad): x={slm_angle_x:.2f}, y={slm_angle_y:.2f}', (10, panel_y + 75),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)
    
    # PID parameters display
    cv2.putText(frame, f'PID: Kp={KP:.2f}, Ki={KI:.2f}, Kd={KD:.2f}', (10, panel_y + 95),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)
    cv2.putText(frame, f'MaxStep={MAX_STEP_PER_FRAME_MRAD:.1f}mrad, Int_X={integral_x:.2f}, Int_Y={integral_y:.2f}', 
                (10, panel_y + 115),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
    cv2.putText(frame, f'SLM {"ON" if slm_updates_enabled else "PAUSED"}  FL={FOCAL_LENGTH_MM}mm', 
                (10, panel_y + 135),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 255, 200), 1)
    
    # Error trend display (optional mini-graph)
    if len(error_history_x) > 1:
        graph_x = frame_width - 150
        graph_y = frame_height - 80
        graph_w = 140
        graph_h = 70
        
        # Draw graph background
        cv2.rectangle(frame, (graph_x, graph_y), (graph_x + graph_w, graph_y + graph_h), 
                     (50, 50, 50), -1)
        cv2.rectangle(frame, (graph_x, graph_y), (graph_x + graph_w, graph_y + graph_h), 
                     (100, 100, 100), 1)
        
        # Draw zero line
        cv2.line(frame, (graph_x, graph_y + graph_h//2), 
                (graph_x + graph_w, graph_y + graph_h//2), (80, 80, 80), 1)
        
        # Scale and draw error history
        max_err = max(2.0, max(abs(e) for e in error_history_x))  # At least ±2 degrees scale
        for i in range(1, len(error_history_x)):
            x1 = graph_x + int((i-1) * graph_w / len(error_history_x))
            x2 = graph_x + int(i * graph_w / len(error_history_x))
            y1 = graph_y + graph_h//2 - int(error_history_x[i-1] * graph_h / (2*max_err))
            y2 = graph_y + graph_h//2 - int(error_history_x[i] * graph_h / (2*max_err))
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 255), 1)
        
        cv2.putText(frame, 'X Error Trend', (graph_x + 5, graph_y - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)

    cv2.imshow('Face Tracking + PID Control', frame)

    # Keys
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('h'):
        FOV_HORIZONTAL = min(FOV_HORIZONTAL + 5, 180)
        print(f"FOV水平: {FOV_HORIZONTAL}°")
    elif key == ord('g'):
        FOV_HORIZONTAL = max(FOV_HORIZONTAL - 5, 1)
        print(f"FOV水平: {FOV_HORIZONTAL}°")
    elif key == ord('v'):
        FOV_VERTICAL = min(FOV_VERTICAL + 5, 180)
        print(f"FOV垂直: {FOV_VERTICAL}°")
    elif key == ord('b'):
        FOV_VERTICAL = max(FOV_VERTICAL - 5, 1)
        print(f"FOV垂直: {FOV_VERTICAL}°")
    
    # PID tuning keys
    elif key == ord('['):  # decrease Kp
        KP = max(0.05, KP - 0.05)
        print(f"Kp -> {KP:.2f}")
    elif key == ord(']'):  # increase Kp
        KP = min(2.0, KP + 0.05)
        print(f"Kp -> {KP:.2f}")
    elif key == ord('i'):  # decrease Ki
        KI = max(0.0, KI - 0.02)
        print(f"Ki -> {KI:.2f}")
    elif key == ord('o'):  # increase Ki
        KI = min(1.0, KI + 0.02)
        print(f"Ki -> {KI:.2f}")
    elif key == ord('k'):  # decrease Kd
        KD = max(0.0, KD - 0.05)
        print(f"Kd -> {KD:.2f}")
    elif key == ord('l'):  # increase Kd
        KD = min(2.0, KD + 0.05)
        print(f"Kd -> {KD:.2f}")
    
    elif key == ord(','):  # decrease max step
        MAX_STEP_PER_FRAME_MRAD = max(0.5, MAX_STEP_PER_FRAME_MRAD - 0.5)
        print(f"MaxStep -> {MAX_STEP_PER_FRAME_MRAD:.1f} mrad")
    elif key == ord('.'):  # increase max step
        MAX_STEP_PER_FRAME_MRAD = min(20.0, MAX_STEP_PER_FRAME_MRAD + 0.5)
        print(f"MaxStep -> {MAX_STEP_PER_FRAME_MRAD:.1f} mrad")
    elif key == ord('r'):  # reset SLM to center and clear PID state
        slm_angle_x = 0.0
        slm_angle_y = 0.0
        reset_pid_state()
        try:
            _ = generate_and_load_fresnel_lens(slm, focal_length_mm=FOCAL_LENGTH_MM,
                                               angle_x_mrad=0.0, angle_y_mrad=0.0)
            print("Reset SLM to center and cleared PID state")
        except Exception as e:
            print("Reset SLM error:", e)
    elif key == ord('c'):  # clear integral terms only
        integral_x = 0.0
        integral_y = 0.0
        print("Cleared integral terms")
    elif key == ord('x'):  # flip X sign
        ANGLE_SIGN_X *= -1.0
        print(f"ANGLE_SIGN_X -> {ANGLE_SIGN_X:+.0f}")
    elif key == ord('y'):  # flip Y sign
        ANGLE_SIGN_Y *= -1.0
        print(f"ANGLE_SIGN_Y -> {ANGLE_SIGN_Y:+.0f}")
    elif key == ord('s'):  # toggle search mode
        search_mode = True
        print("Force SEARCH mode when no face")
    elif key == ord('p'):  # pause/resume SLM updates
        slm_updates_enabled = not slm_updates_enabled
        print("SLM updates:", "ENABLED" if slm_updates_enabled else "PAUSED")

print("\n程序结束")
print(f"Final PID parameters: Kp={KP:.2f}, Ki={KI:.2f}, Kd={KD:.2f}")
cap.release()
cv2.destroyAllWindows()

按 'q' 退出程序
相机FOV设置: 4° (水平) x 3° (垂直)
分辨率: 1024 x 576

=== PID控制键盘快捷键 ===
P增益: [ / ] 调整Kp
I增益: i / o 调整Ki
D增益: k / l 调整Kd
其他: ,/. 调整MAX_STEP, r复位SLM+PID, x/y翻转轴符号
     s切换搜索, p暂停SLM更新, c清除积分项

初始PID参数: Kp=0.40, Ki=0.20, Kd=0.01
Reset SLM to center and cleared PID state

程序结束
Final PID parameters: Kp=0.40, Ki=0.20, Kd=0.01
